In [1]:
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import wikipedia

In [2]:
def tokenize(text, add_stop_token = True):
    tokens = []
    for letter in text:
        if letter in TOKEN_DICT:
            tokens.append(TOKEN_DICT[letter])
        else:
            # UNK token
            tokens.append(len(TOKEN_DICT))
            
    # STOP token
    if add_stop_token:
        tokens.append(len(TOKEN_DICT) + 1)
    return torch.tensor(tokens)

def read_tokens_from_file(filename):
    with open(filename, "r") as file:
        text = file.read()
        return tokenize(text)

In [3]:
def get_giga_token_counts(verbose = False):
    mega_text = ""
    for directory in ["AA", "AB", "AC"]:
        for digit1 in range(10):
            for digit2 in range(10):
                filename = f"text/{directory}/wiki_{digit1}{digit2}"
                try:
                    with open(filename, "r") as file:
                        text = file.read()
                        mega_text += text
                except FileNotFoundError:
                    if verbose:
                        print(f"File {filename} not found")
                    
    return mega_text

In [4]:
mega_text = get_giga_token_counts()
token_counts = pd.Series(list(mega_text)).value_counts()
token_counts = token_counts[token_counts > 300]
TOKEN_DICT = {e:i for i, e in enumerate(token_counts.index)}
REVERSE_TOKEN_DICT = {i:e for i, e in enumerate(token_counts.index)}

In [5]:
D_MODEL = 96
DK = 32
N_HEADS = 16
CONTEXT_SIZE = 24
MLP_HIDDEN_LAYER_SIZE = 192
N_DISTINCT_TOKENS = len(TOKEN_DICT) + 2 # The + 2 is for UNK and STOP tokens
N_LAYERS = 20
P_DROPOUT = 0.1

In [6]:
def create_mask_matrix(dim):
    neginf = (torch.tensor(-1) / torch.tensor(0)).item()
    matrix = torch.zeros(dim, dim)
    for i in range(dim):
        for j in range(i + 1, dim):
            matrix[i][j] = neginf
            
    return matrix
MASK_MATRIX = create_mask_matrix(CONTEXT_SIZE).unsqueeze(0)

In [7]:
def positional_embeddings(d_model, context_size):
    a = torch.zeros(context_size, d_model)
    for pos in range(context_size):
        for j in range(d_model):
            if j % 2 == 0:
                a[pos][j] = math.sin(pos / 10000 ** (j / d_model))
            else:
                a[pos][j] = math.cos(pos / 10000 ** ((j - 1) / d_model))
    return a
                
def positional_embeddings2(d_model, max_len):
    # Source: https://nlp.seas.harvard.edu/annotated-transformer/#positional-encoding
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
    )
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

# These are the exact same way of getting positional embeddings, barring floating-point precision issues.

In [8]:
class AttentionHead(nn.Module):
    def __init__(self):
        super(AttentionHead, self).__init__()
        self.Wk = nn.Linear(D_MODEL, DK, bias = False)
        self.Wq = nn.Linear(D_MODEL, DK, bias = False)
        self.Wv = nn.Linear(D_MODEL, DK, bias = False)
        self.dropout = nn.Dropout(p = P_DROPOUT)
        
        # The weights are initialized with Kaiming uniform by default, which is fine for now
        
    def forward(self, x):
        K = self.Wk(x)
        Q = self.Wq(x)
        V = self.Wv(x)
        
        QKT = torch.matmul(Q, K.transpose(dim0 = 1, dim1 = 2))
        QKT += MASK_MATRIX
        pattern = torch.softmax(QKT / math.sqrt(DK), dim = 2)
        pattern = self.dropout(pattern)
        return torch.matmul(pattern, V)
    
class AttentionBlock(nn.Module):
    def __init__(self):
        super(AttentionBlock, self).__init__()
        self.heads = nn.ModuleList([AttentionHead() for i in range(N_HEADS)])
        self.Wo = nn.Linear(DK * N_HEADS, D_MODEL, bias = False)
        
    def forward(self, x):
        return self.Wo(
                torch.concat(
                    [head(x) for head in self.heads],
                        dim = 2))
    
class MLPBlock(nn.Module):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.layer1 = nn.Linear(D_MODEL, MLP_HIDDEN_LAYER_SIZE)
        self.activation1 = nn.GELU()
        self.layer2 = nn.Linear(MLP_HIDDEN_LAYER_SIZE, D_MODEL)
        # The nn.Linear layers are initialized with Kaiming uniform initialization by default, which is fine
        
    def forward(self, x):
        x1 = self.layer1(x)
        x2 = self.activation1(x1)
        x3 = self.layer2(x2)
        return x3
    
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        
        # Embedding, positional encoding, and dropout
        self.embedding = nn.Embedding(N_DISTINCT_TOKENS, D_MODEL)
        self.positional_embedding = positional_embeddings(D_MODEL, CONTEXT_SIZE).unsqueeze(0)
        self.first_dropout = nn.Dropout(p = P_DROPOUT)

        # Attention
        self.attention_blocks = nn.ModuleList()
        for i in range(N_LAYERS):
            layer_norm = nn.LayerNorm(D_MODEL)
            attention = AttentionBlock()
            dropout = nn.Dropout(P_DROPOUT)
            module = nn.Sequential(layer_norm, attention, dropout)
            self.attention_blocks.append(module)
            
        # MLP
        self.mlps = nn.ModuleList()
        for i in range(N_LAYERS):
            layer_norm = nn.LayerNorm(D_MODEL)
            mlp = MLPBlock()
            dropout = nn.Dropout(P_DROPOUT)
            module = nn.Sequential(layer_norm, mlp, dropout)
            self.mlps.append(module)
            
        # Final layer norm and linear
        self.final_layer_norm = nn.LayerNorm(D_MODEL)
        self.final_linear = nn.Linear(D_MODEL, N_DISTINCT_TOKENS)
            
    def forward(self, x):
        # Step 1: Embedding and positional encoding
        x = self.embedding(x) + self.positional_embedding
        
        # Step 2: First dropout
        x = self.first_dropout(x)
        
        # Step 3: All the blocks
        for i in range(N_LAYERS):
            x = x + self.attention_blocks[i](x)
            x = x + self.mlps[i](x)
            
        # Step 4: Final layer norm
        x = self.final_layer_norm(x)
        
        # Step 5: Final linear layer to get predictions
        x = self.final_linear(x)
        
        # Step 6: Return
        return x

In [9]:
def no_peeking_test():
    my_model = Transformer()
    my_model.eval()
    for pos in range(CONTEXT_SIZE):
        x = torch.arange(0, CONTEXT_SIZE * 2, 2).unsqueeze(0)
        output1 = my_model(x)
        for disturb_pos in range(pos + 1, CONTEXT_SIZE):
            x[0][disturb_pos] += 1
            output2 = my_model(x)
            (output1[0,:pos + 1] == output2[0,:pos + 1]).all()
            
no_peeking_test()

In [10]:
def nonzero_gradient_test():
    my_model = Transformer()
    x = torch.arange(0, 26, 2)
    out = my_model(x)
    loss = out.mean()
    loss.backward()
    print([e.grad for e in my_model.parameters()])
    
# nonzero_gradient_test()

In [11]:
def nonzero_gradient_test():
    my_model = Transformer()

    # 1) turn off dropout
    my_model.eval()

    # 2) (optional) fix RNG
    torch.manual_seed(1234)
    torch.cuda.manual_seed_all(1234)
    torch.use_deterministic_algorithms(True)

    # 3) run forward & backward
    x = torch.arange(0, 26, 2)           # your token indices
    out = my_model(x)
    loss = out.mean()
    loss.backward()

    # 4) inspect gradients
    for name, param in my_model.named_parameters():
        if param.requires_grad:
            g = param.grad
            print(f"{name:30s} → norm = {g.norm().item():.3e}")
        else:
            g = param.grad
            print(f"DOES NOT REQUIRE GRAD: {name:30s} → norm = {g.norm().item():.3e}")
    
# nonzero_gradient_test()

In [12]:
def get_dataset(tokens):
    Xs = []
    ys = []
    for i in range(0, len(tokens) - CONTEXT_SIZE - 1, CONTEXT_SIZE):
        X = tokens[i     : i + CONTEXT_SIZE    ]
        y = tokens[i + 1 : i + CONTEXT_SIZE + 1]
        Xs.append(X)
        ys.append(y)

    X = torch.stack(Xs)
    y = torch.stack(ys)
    return X, y

In [13]:
def train_one_epoch(model, optimizer, loss_fn, tokens, batch_size = 32):
    model.train()
    t0 = time.time()
    X_all, y_all = get_dataset(tokens)
    loss_vals = []
    for i in range(0, X_all.shape[0], batch_size):
        X = X_all[i : i + batch_size]
        y = y_all[i : i + batch_size]
        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1, N_DISTINCT_TOKENS), y.view(-1))
        loss.backward()
        optimizer.step()
        
        if i == 0:
            for param in model.parameters():
                print(param.grad)
        
        optimizer.zero_grad()
        loss_vals.append(loss.item())
        t1 = time.time()
        
    batch_loss = torch.mean(torch.tensor(loss_vals))
    print(f"{i + 1} samples evaluated, time={int(t1-t0)}, loss = {batch_loss:.4f}")

In [14]:
def run_epoch_on_file(model, optimizer, loss_fn, filename, batch_size = 32):
    train_one_epoch(model, optimizer, loss_fn, read_tokens_from_file(filename), batch_size)

In [15]:
my_model = Transformer()
for digit1 in range(1):
    for digit2 in range(10):
        run_epoch_on_file(my_model,
                        optimizer = torch.optim.Adam(my_model.parameters()),
                        loss_fn = nn.CrossEntropyLoss(),
                        filename = f"text/AA/wiki_{digit1}{digit2}",
                        batch_size = 2048)

40961 samples evaluated, time=1143, loss = 3.7329
40961 samples evaluated, time=1043, loss = 2.9423
40961 samples evaluated, time=970, loss = 2.7601
43009 samples evaluated, time=1074, loss = 2.6713
43009 samples evaluated, time=974, loss = 2.6222
43009 samples evaluated, time=1038, loss = 2.5703
43009 samples evaluated, time=1028, loss = 2.4927
40961 samples evaluated, time=1062, loss = 2.4342
40961 samples evaluated, time=1163, loss = 2.3851
43009 samples evaluated, time=1052, loss = 2.3714


In [19]:
def predict_next_token(model, text):
    model.eval()
    trimmed = text[-CONTEXT_SIZE:]
    tokens = tokenize(trimmed, add_stop_token = False).unsqueeze(0)
    output = model(tokens)
    logits = output[:,0:1,:].squeeze()
    probs = torch.softmax(logits, dim = 0) # We can do argmax just on the logits if we want, but whatever.
    token_id = torch.argmax(logits).item()
    return REVERSE_TOKEN_DICT[token_id]

In [20]:
def predict_next_tokens(model, text, num_tokens):
    model.eval()
    for i in range(num_tokens):
        text += predict_next_token(model, text)
    
    return text

In [22]:
predict_next_tokens(my_model, "Anarchism is a political ", 100)

'Anarchism is a political  neten etn tntrnenhntnett  h  t h th he   e h  hhttetthtethete ttt tetteehh hheh he h thhhth hh  eet'

In [25]:
def get_test_error(model, loss_fn, tokens, batch_size = 1024):
    model.eval()
    t0 = time.time()
    X_all, y_all = get_dataset(tokens)
    loss_vals = []
    for i in range(0, X_all.shape[0], batch_size):
        X = X_all[i : i + batch_size]
        y = y_all[i : i + batch_size]
        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1, N_DISTINCT_TOKENS), y.view(-1))
        loss_vals.append(loss.item())
        t1 = time.time()
        
    batch_loss = torch.mean(torch.tensor(loss_vals))
    return batch_loss

In [26]:
get_test_error(my_model,
               nn.CrossEntropyLoss(),
               read_tokens_from_file("text/AD/wiki_62"))

tensor(2.2822)

In [42]:
for name, param in my_model.named_parameters():
    moment2 = (param ** 2).mean().item()
    if True and moment2 > 0.5:
        print(name)
        print(param)
        print(tuple(param.shape))
        
    if False and tuple(param.shape) == (198, 96):
        print(moment2)

embedding.weight
Parameter containing:
tensor([[ 0.6801,  0.2517,  0.9339,  ..., -1.1119,  0.2616,  0.5658],
        [ 0.7903, -1.4222, -0.4583,  ...,  0.2672,  0.5408,  0.3611],
        [ 0.2770, -0.0861,  0.0234,  ..., -1.1143, -1.1699, -0.2858],
        ...,
        [-0.5868, -0.8638,  0.1274,  ..., -0.1934, -0.3445,  0.5701],
        [ 1.0681,  1.7862,  1.1129,  ...,  0.2777,  1.4612,  2.3642],
        [-0.8963,  0.8775, -0.5612,  ..., -0.7263, -0.4534,  1.4578]],
       requires_grad=True)
(198, 96)
attention_blocks.0.0.weight
Parameter containing:
tensor([1.0191, 1.0447, 1.0169, 1.0196, 1.0019, 1.0294, 1.0523, 1.0361, 1.0392,
        1.0240, 1.0349, 1.0598, 1.0583, 1.0416, 1.0585, 1.0172, 1.0637, 1.0585,
        1.0595, 1.0721, 1.0241, 1.0661, 0.9983, 1.0762, 1.0386, 1.0618, 1.0699,
        1.1365, 1.1005, 1.0398, 1.0659, 1.0465, 1.0287, 0.9954, 1.0592, 1.0561,
        1.0288, 1.0164, 1.0080, 0.9988, 1.0484, 1.0153, 1.0205, 1.0140, 1.0465,
        1.0078, 1.0156, 1.0239, 1.0240, 

In [ ]:
# OK, clearly the model is really bad right now